In [ ]:
# ========================================
# 1) Connect to Google Drive
# ========================================
#from google.colab import drive
#drive.mount('/content/drive')

#MODEL_DIR = "/content/drive/MyDrive/deep_learning"


Mounted at /content/drive


In [ ]:
# ========================================
# 2) Install required libraries
# ========================================
!pip install -U transformers datasets accelerate evaluate optuna wandb


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 71.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install evaluate

In [ ]:
!pip install -U transformers


In [ ]:
# ========================================
# 3) Import libraries
# ========================================
import os, time, copy, tempfile, inspect
import numpy as np
import pandas as pd
from pathlib import Path
import torch, torch.nn as nn
from torch.quantization import quantize_dynamic
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
import os
import re
import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report


In [ ]:
# Check if GPU is available and select device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# === Load data ===
#test_df = pd.read_csv('/content/drive/MyDrive/deep_learning/val_processed.csv', encoding='latin1')
import pandas as pd

# Navigate from model folder to data folder
current_dir = Path.cwd()  # models/bert-base-uncased/
models_dir = current_dir.parent  # models/
project_root = models_dir.parent  # project root
data_dir = models_dir / 'data'

# Load data
train_df = pd.read_csv(data_dir / 'train_processed.csv', encoding='latin1')
eval_df = pd.read_csv(data_dir / 'val_processed.csv', encoding='latin1')
test_df = pd.read_csv(data_dir / 'test_processed.csv', encoding='latin1')


print(f"Test shape: {test_df.shape}")
# Define label order
ordered_labels = ['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive']
label2id = {lbl:i for i,lbl in enumerate(ordered_labels)}
id2label = {i:lbl for lbl,i in label2id.items()}


In [ ]:
# --------- config ----------
#MODEL_DIR  = "/content/drive/MyDrive/deep_learning"
MODEL_NAME = "bert-base-uncased"
MODEL_DIR  =data_dir


In [ ]:
test_df["label"] = test_df["Sentiment"].map(label2id)

In [ ]:
def clean_for_cardiffnlp(text):
    if pd.isnull(text):
        return ""

    tokens = []
    for t in text.split(" "):
        if t.startswith("@") and len(t) > 1:
            tokens.append("@user")
        elif t.startswith("http"):
            tokens.append("http")
        else:
            tokens.append(t)
    text = " ".join(tokens)

    # Normalize common COVID variants to "covid"
    text = re.sub(r"\b(coronaviruspandemic|covid[_\s-]*2019|covid[_\s-]*19|covid2019|coronavirus2019|coronavirus|corona)\b", "covid", text, flags=re.IGNORECASE)

    # Decode HTML entities
    text = html.unescape(text)

    # Normalize whitespace and repeated punctuation (optional)
    text = re.sub(r"\s+", " ", text).strip()
    text = re.sub(r"(\.\s*){2,}", ". ", text)
    text = re.sub(r"([!?]){2,}", r"\1", text)
    text = re.sub(r"(\?\s+){2,}", "?", text)
    text = re.sub(r"(\!\s+){2,}", "!", text)

    return text


In [ ]:
BASE_CHECKPOINT = "bert-base-uncased"
LABEL_COL = "label"
BATCH_SIZE = 128
NUM_WORKERS = 2
PIN_MEMORY = True


# ==== tokenizer + collator ====
tokenizer = AutoTokenizer.from_pretrained(BASE_CHECKPOINT, use_fast=True)
collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ==== Class Dataset ====
class TextClsDataset(Dataset):
    def __init__(self, df, text_col, label_col):
        self.texts = df[text_col].astype(str).tolist()
        self.labels = df[label_col].astype(int).tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = tokenizer(
            self.texts[idx],
            truncation=True,
            max_length=128,   # לשנות אם צריך
            return_tensors=None
        )
        enc["labels"] = self.labels[idx]
        return enc

# prepere datasets
test_ds_clean = TextClsDataset(test_df, text_col="ProcessedTweet",        label_col=LABEL_COL)
#test_ds_light = TextClsDataset(test_df, text_col="ProcessedTweet_light",  label_col=LABEL_COL)

# DataLoaders
test_loader_clean = DataLoader(
    test_ds_clean,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    collate_fn=collator
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
# Use the same model class as in training
class ConfigurableBertModel(nn.Module):
    def __init__(self, model_name: str, num_labels: int, dropout_rate: float = 0.1):
        super(ConfigurableBertModel, self).__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.encoder.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None):
        outputs = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0]  # CLS token
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return type('ModelOutput', (), {'logits': logits})()



In [ ]:
#MODEL_DIR = "/content/drive/MyDrive/deep_learning"
MODEL_DIR  =data_dir

# Map -> preprocessing
MODEL_PREPROC = {
    "best_BERT_base_uncasedmodel2.pt": "clean",
}

# Evaluation function
def evaluate_model(model, dataloader, device):
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for batch in dataloader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Classification report
    print("Classification Report:")
    print(classification_report(
        all_labels, all_preds,
        target_names=['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive']
    ))

    return {
        "accuracy": accuracy_score(all_labels, all_preds),
        "f1": f1_score(all_labels, all_preds, average='macro'),
        "precision": precision_score(all_labels, all_preds, average='macro'),
        "recall": recall_score(all_labels, all_preds, average='macro')
    }


# Evaluate all saved models
def evaluate_all_models(model_dir, model_preproc_map, test_loader_clean, device):
    results = {}

    for model_file, preproc_type in model_preproc_map.items():
        print(f"\n=== Evaluating {model_file} ({preproc_type}) ===")

        model_path = os.path.join(model_dir, model_file)

        # IMPORTANT: use ConfigurableBertModel, same as in training
        model = ConfigurableBertModel(
            model_name="bert-base-uncased",
            num_labels=5,
            dropout_rate=0.2  # you can change if needed
        )
        model.load_state_dict(torch.load(model_path, map_location=device))
        model.to(device)

        if preproc_type == "clean":
            test_loader = test_loader_clean
        elif preproc_type == "light":
            test_loader = test_loader_light
        else:
            raise ValueError(f"Unknown preprocessing type: {preproc_type}")

        metrics = evaluate_model(model, test_loader, device)
        results[model_file] = metrics

    print("\n=== Summary of All Models ===")
    for model_file, metrics in results.items():
        print(f"{model_file}: "
              f"Acc={metrics['accuracy']:.4f}, "
              f"F1={metrics['f1']:.4f}, "
              f"Precision={metrics['precision']:.4f}, "
              f"Recall={metrics['recall']:.4f}")

    return results


In [ ]:
results = evaluate_all_models(MODEL_DIR, MODEL_PREPROC, test_loader_clean, device)


=== Evaluating best_BERT_base_uncasedmodel2.pt (clean) ===


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Classification Report:
                    precision    recall  f1-score   support

Extremely Negative       0.86      0.89      0.87       592
          Negative       0.85      0.85      0.85      1041
           Neutral       0.94      0.86      0.90       619
          Positive       0.83      0.83      0.83       947
Extremely Positive       0.85      0.88      0.86       599

          accuracy                           0.86      3798
         macro avg       0.87      0.86      0.86      3798
      weighted avg       0.86      0.86      0.86      3798


=== Summary of All Models ===
best_BERT_base_uncasedmodel2.pt: Acc=0.8597, F1=0.8640, Precision=0.8655, Recall=0.8637


In [ ]:
BASE_CHECKPOINT = "cardiffnlp/twitter-roberta-base-sentiment-latest"
LABEL_COL = "label"
BATCH_SIZE = 128
NUM_WORKERS = 2
PIN_MEMORY = True


# ==== tokenizer + collator ====
tokenizer = AutoTokenizer.from_pretrained(BASE_CHECKPOINT, use_fast=True)
collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ==== Class Dataset ====
class TextClsDataset(Dataset):
    def __init__(self, df, text_col, label_col):
        self.texts = df[text_col].astype(str).tolist()
        self.labels = df[label_col].astype(int).tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        enc = tokenizer(
            self.texts[idx],
            truncation=True,
            max_length=128,
            return_tensors=None
        )
        enc["labels"] = self.labels[idx]
        return enc

# prepere datasets
test_ds_clean = TextClsDataset(test_df, text_col="ProcessedTweet",        label_col=LABEL_COL)

# DataLoaders
test_loader_clean = DataLoader(
    test_ds_clean,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=PIN_MEMORY,
    collate_fn=collator
)



config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
import os
import re
import torch
import torch.nn as nn
from pathlib import Path
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, DataCollatorWithPadding
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# ==============================
# Config
# ==============================
MODEL_DIR  =data_dir

BASE_CHECKPOINT = "bert-base-uncased"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluate these saved heads (your files)
MODEL_FILES = [
    "HF_best_model_stage3.pt",
]

ORDERED_LABELS = ['Extremely Negative', 'Negative', 'Neutral', 'Positive', 'Extremely Positive']
LABEL2ID = {l: i for i, l in enumerate(ORDERED_LABELS)}

# ==============================
# Tokenizer (+ special tokens)
# ==============================
tokenizer = AutoTokenizer.from_pretrained(BASE_CHECKPOINT, use_fast=True)
specials = {"additional_special_tokens": ["<httpurl>", "<user>", "<hashtag>", "<emoji>"]}
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
collator = DataCollatorWithPadding(tokenizer=tokenizer)


# ==============================
# Preprocessing (match training)
#   - URLs  -> <httpurl>
#   - @user -> <user>
#   - hashtags -> prefix with <hashtag> (keep the hashtag token)
#   - emojis -> prefix with <emoji> before the emoji cluster
# ==============================
_EMOJI_RE = re.compile(
    r'['
    r'\U0001F1E0-\U0001F1FF'  # flags
    r'\U0001F300-\U0001F5FF'  # symbols & pictographs
    r'\U0001F600-\U0001F64F'  # emoticons
    r'\U0001F680-\U0001F6FF'  # transport & map
    r'\U0001F700-\U0001F77F'
    r'\U0001F780-\U0001F7FF'
    r'\U0001F800-\U0001F8FF'
    r'\U0001F900-\U0001F9FF'
    r'\U0001FA00-\U0001FA6F'
    r'\U0001FA70-\U0001FAFF'
    r'\u2600-\u26FF\u2700-\u27BF'
    r']+'
)

def preprocess_with_markers(t: str) -> str:
    if not isinstance(t, str):
        return ""
    t = re.sub(r'https?://\S+', '<httpurl>', t)
    t = re.sub(r'@\w+', '<user>', t)
    # add "<hashtag> " before hashtags that don't already have it
    t = re.sub(r'(?<!<hashtag>)#\w+', lambda m: f"<hashtag> {m.group(0)}", t)
    # add "<emoji> " before emoji clusters
    t = _EMOJI_RE.sub(lambda m: f"<emoji> {m.group(0)}", t)
    t = re.sub(r'\s+', ' ', t).strip()
    return t

# ==============================
# Dataset
# ==============================
class TDataset(Dataset):
    def __init__(self, texts, labels, max_len=128):
        self.texts = texts
        self.labels = labels
        self.max_len = max_len
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, i):
        enc = tokenizer(
            self.texts[i],
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt"
        )
        item = {k: v.squeeze(0) for k, v in enc.items()}
        item["labels"] = torch.tensor(self.labels[i], dtype=torch.long)
        return item

# ==============================
# Model (same head you trained)
# ==============================
class BertWithDropout(nn.Module):
    def __init__(self, model_name, num_labels=5, dropout_rate=0.2):
        super().__init__()
        self.backbone = AutoModel.from_pretrained(model_name)
        self.dropout  = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_labels)
    def forward(self, input_ids, attention_mask=None, labels=None, token_type_ids=None):
        outputs = self.backbone(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        cls = outputs.last_hidden_state[:, 0, :]
        logits = self.classifier(self.dropout(cls))
        return {"logits": logits}

# ==============================
# Evaluation loop
# ==============================
def evaluate_model(model, dataloader):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch in dataloader:
            ids   = batch["input_ids"].to(DEVICE)
            mask  = batch.get("attention_mask")
            ttids = batch.get("token_type_ids")  # BERT supplies these
            if mask  is not None:  mask  = mask.to(DEVICE)
            if ttids is not None: ttids = ttids.to(DEVICE)

            labels = batch["labels"].cpu().numpy()
            logits = model(ids, attention_mask=mask, token_type_ids=ttids)["logits"]
            preds  = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_labels.extend(labels)

    print(classification_report(all_labels, all_preds, target_names=ORDERED_LABELS, digits=4))
    return {
        "accuracy":  accuracy_score(all_labels, all_preds),
        "f1":        f1_score(all_labels, all_preds, average='macro'),
        "precision": precision_score(all_labels, all_preds, average='macro'),
        "recall":    recall_score(all_labels, all_preds, average='macro'),
    }

# ==============================
# Prepare test data and run
#   Expects: test_df with columns ["OriginalTweet","Sentiment"]
# ==============================
test_df["label"] = test_df["Sentiment"].map(LABEL2ID)

texts  = [preprocess_with_markers(x) for x in test_df["OriginalTweet"].astype(str)]
labels = test_df["label"].tolist()
ds = TDataset(texts, labels)
dl = DataLoader(ds, batch_size=128, shuffle=False, collate_fn=collator)

for model_file in MODEL_FILES:
    print(f"\n=== Evaluating {model_file} ===")
    model = BertWithDropout("bert-base-uncased", num_labels=len(ORDERED_LABELS), dropout_rate=0.2)


    state = torch.load(os.path.join(MODEL_DIR, model_file), map_location=DEVICE)
    model.load_state_dict(state, strict=True)
    model.to(DEVICE)

    metrics = evaluate_model(model, dl)
    print(f"Acc={metrics['accuracy']:.4f}, F1(macro)={metrics['f1']:.4f}, "
          f"Precision={metrics['precision']:.4f}, Recall={metrics['recall']:.4f}")


=== Evaluating HF_best_model_stage3.pt ===
                    precision    recall  f1-score   support

Extremely Negative     0.8961    0.8885    0.8923       592
          Negative     0.8814    0.8492    0.8650      1041
           Neutral     0.7828    0.8966    0.8358       619
          Positive     0.8552    0.8416    0.8483       947
Extremely Positive     0.9048    0.8564    0.8799       599

          accuracy                         0.8623      3798
         macro avg     0.8640    0.8665    0.8643      3798
      weighted avg     0.8647    0.8623    0.8627      3798

Acc=0.8623, F1(macro)=0.8643, Precision=0.8640, Recall=0.8665


# Post-training Dynamic Quantization

In [ ]:
DEVICE = torch.device("cpu")
# Check
print(f"Using device: {DEVICE}")

Using device: cpu


In [ ]:
# ======================
# Step 1: Robust loader for  Bert
# ======================

import torch
import torch.nn as nn
from torch.quantization import quantize_dynamic
import os

# ======================
# 1) Load your best Stage 3 model
# ======================
# --------- config ----------
MODEL_DIR  =data_dir
MODEL_REGISTRY = {
    "HF_best_model_stage3.pt":            ("bert-base-uncased",                                      BertWithDropout),
    "best_BERT_base_uncasedmodel2.pt":    ("bert-base-uncased",                                      ConfigurableBertModel),
}

def _state_dict_size_mb(model) -> float:
    """Compute saved state_dict size in MB."""
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp:
        torch.save(model.state_dict(), tmp.name)
        mb = os.path.getsize(tmp.name) / (1024**2)
    os.remove(tmp.name)
    return mb

def load_model_for_file(model_file: str, num_labels: int = 5, dropout: float = 0.2):
    """Instantiate the right class/backbone and load weights to CPU."""
    if model_file not in MODEL_REGISTRY:
        raise ValueError(f"Unknown model file: {model_file}")

    backbone, cls = MODEL_REGISTRY[model_file]
    model = cls(backbone, num_labels=num_labels, dropout_rate=dropout)
    state = torch.load(os.path.join(MODEL_DIR, model_file), map_location="cpu")
    model.load_state_dict(state, strict=True)
    model.eval().cpu()
    return model, backbone

print("Model                              | Backbone                                    | #Params        | state_dict (MB)")
print("-"*110)
for model_file in MODEL_REGISTRY.keys():
    model, backbone = load_model_for_file(model_file, num_labels=5, dropout=0.2)
    param_count = sum(p.numel() for p in model.parameters())
    size_mb = _state_dict_size_mb(model)
    print(f"{model_file:34s} | {backbone:42s} | {param_count:>13,d} | {size_mb:>14.2f}")

Model                              | Backbone                                    | #Params        | state_dict (MB)
--------------------------------------------------------------------------------------------------------------
HF_best_model_stage3.pt            | bert-base-uncased                          |   109,486,085 |         417.73
best_BERT_base_uncasedmodel2.pt    | bert-base-uncased                          |   109,486,085 |         417.73


In [ ]:
# ======================
# 2) Apply Post-Training Dynamic Quantization for BOTH BERT models
# ======================
from torch.quantization import quantize_dynamic

TARGET_FILES = [
    "HF_best_model_stage3.pt",
    "best_BERT_base_uncasedmodel2.pt",
]

OUT_DIR = os.path.join(MODEL_DIR, "quantized_int8")
os.makedirs(OUT_DIR, exist_ok=True)

for fname in TARGET_FILES:
    print(f"\n=== Quantizing: {fname} ===")
    model, backbone = load_model_for_file(fname, num_labels=len(ordered_labels), dropout=0.2)

    # sizes before/after
    size_fp32 = _state_dict_size_mb(model)
    quantized_model = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)  # BERT: ok to quantize all Linear
    size_int8 = _state_dict_size_mb(quantized_model)

    print("Quantization complete. Model is ready for inference.")
    print(f"State dict size: FP32={size_fp32:.2f} MB -> INT8={size_int8:.2f} MB")

    # ======================
    # 3) Save quantized model (unique name per file)
    # ======================
    quant_path = os.path.join(OUT_DIR, f"{Path(fname).stem}_quantized_int8_sd.pt")
    torch.save(quantized_model.state_dict(), quant_path)
    print(f"Quantized model saved to: {quant_path}")



=== Quantizing: HF_best_model_stage3.pt ===
Quantization complete. Model is ready for inference.
State dict size: FP32=417.73 MB -> INT8=173.09 MB
Quantized model saved to: /content/drive/MyDrive/deep_learning/quantized_int8/HF_best_model_stage3_quantized_int8_sd.pt

=== Quantizing: best_BERT_base_uncasedmodel2.pt ===
Quantization complete. Model is ready for inference.
State dict size: FP32=417.73 MB -> INT8=173.09 MB
Quantized model saved to: /content/drive/MyDrive/deep_learning/quantized_int8/best_BERT_base_uncasedmodel2_quantized_int8_sd.pt


In [ ]:
# ==== INT8 quantization + evaluation for the TWO BERT models ====
import os, tempfile, torch, torch.nn as nn
from torch.quantization import quantize_dynamic
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# Assumes these are already defined earlier in your notebook:
# - MODEL_DIR
# - ordered_labels
# - load_model_for_file(model_file, num_labels, dropout)

# ---------- utilities ----------
import inspect
@torch.inference_mode()
def eval_int8_cpu(model, dataloader, label_names=None):
    """Evaluate on CPU and print per-class report (passes token_type_ids only if supported)."""
    model.eval(); model.cpu()
    # detect which args the model.forward supports
    fwd_params = set(inspect.signature(model.forward).parameters.keys())

    all_preds, all_labels = [], []
    for batch in dataloader:
        ids   = batch["input_ids"]
        mask  = batch.get("attention_mask")
        ttids = batch.get("token_type_ids")

        # build kwargs safely
        kwargs = {"input_ids": ids, "attention_mask": mask}
        if ttids is not None and "token_type_ids" in fwd_params:
            kwargs["token_type_ids"] = ttids

        out = model(**kwargs)
        logits = out["logits"] if isinstance(out, dict) else out.logits
        preds  = torch.argmax(logits, dim=1).cpu().numpy()
        labels = batch.get("labels", batch.get("label")).cpu().numpy()

        all_preds.extend(preds); all_labels.extend(labels)

    # report
    print(classification_report(all_labels, all_preds, target_names=label_names, digits=4))
    metrics = {
        "accuracy":  accuracy_score(all_labels, all_preds),
        "f1":        f1_score(all_labels, all_preds, average="macro"),
        "precision": precision_score(all_labels, all_preds, average="macro", zero_division=0),
        "recall":    recall_score(all_labels, all_preds, average="macro"),
    }
    print(f"Accuracy={metrics['accuracy']:.4f} | F1(macro)={metrics['f1']:.4f} | "
          f"Precision(macro)={metrics['precision']:.4f} | Recall(macro)={metrics['recall']:.4f}")
    return metrics


# ---------- build a BERT-specific test loader (tokenizer must match BERT) ----------
tok_bert = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
coll_bert = DataCollatorWithPadding(tokenizer=tok_bert)

class _EvalBertDS(Dataset):
    def __init__(self, df, text_col="ProcessedTweet", label_col="label", max_len=128):
        self.texts  = df[text_col].astype(str).tolist()
        self.labels = df[label_col].astype(int).tolist()
        self.max_len = max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        enc = tok_bert(self.texts[i], truncation=True, max_length=self.max_len)
        enc["labels"] = self.labels[i]
        return enc

dl_bert = DataLoader(_EvalBertDS(test_df), batch_size=128, shuffle=False, collate_fn=coll_bert)

# ---------- quantize + evaluate each BERT model ----------
bert_files = ["HF_best_model_stage3.pt", "best_BERT_base_uncasedmodel2.pt"]

for MODEL_FILE in ["HF_best_model_stage3.pt", "best_BERT_base_uncasedmodel2.pt"]:
    model, _ = load_model_for_file(MODEL_FILE, num_labels=len(ordered_labels), dropout=0.2)
    q_model = quantize_dynamic(model, {nn.Linear}, dtype=torch.qint8)
    print(f"\nINT8 eval: {MODEL_FILE}")
    _ = eval_int8_cpu(q_model, dl_bert, label_names=ordered_labels)  # או dl שמתאים ל-BERT




INT8 eval: HF_best_model_stage3.pt
                    precision    recall  f1-score   support

Extremely Negative     0.9124    0.8091    0.8577       592
          Negative     0.8351    0.8223    0.8287      1041
           Neutral     0.7632    0.8853    0.8197       619
          Positive     0.8247    0.8194    0.8220       947
Extremely Positive     0.8591    0.8447    0.8519       599

          accuracy                         0.8333      3798
         macro avg     0.8389    0.8362    0.8360      3798
      weighted avg     0.8366    0.8333    0.8337      3798

Accuracy=0.8333 | F1(macro)=0.8360 | Precision(macro)=0.8389 | Recall(macro)=0.8362

INT8 eval: best_BERT_base_uncasedmodel2.pt
                    precision    recall  f1-score   support

Extremely Negative     0.8496    0.8970    0.8726       592
          Negative     0.8615    0.8425    0.8519      1041
           Neutral     0.9262    0.8514    0.8872       619
          Positive     0.8255    0.8194    0.8225   

# unstructured pruning

In [ ]:
# ========================================
# Global L1 Unstructured Pruning for BERT (clean E2E)
# ========================================
import os, tempfile, torch, torch.nn as nn
from torch.nn.utils import prune

BEST_FILE = os.path.join(MODEL_DIR, "HF_best_model_stage3.pt")
PRUNED_SD = os.path.join(MODEL_DIR, "HF_best_model_stage3_pruned40_sd.pt")

# --- 1) Loader for your trained model (FP32) ---
def load_fp32_model_state(path, num_labels):
    model = BertWithDropout("bert-base-uncased", num_labels=num_labels, dropout_rate=0.2)
    state = torch.load(path, map_location="cpu")
    model.load_state_dict(state, strict=True)
    model.eval().cpu()
    return model

# --- 2) Utilities for pruning and reporting ---
def collect_linear_params(model, prune_classifier=False):
    """Collect (module, 'weight') pairs for nn.Linear layers.
       If prune_classifier=False, keep the final classifier in FP32."""
    params = []
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear):
            if not prune_classifier and name.endswith("classifier"):
                continue
            params.append((module, "weight"))
    return params

def apply_global_l1(model, amount=0.4, prune_classifier=False):
    """Global unstructured L1 pruning across selected Linear weights."""
    targets = collect_linear_params(model, prune_classifier=prune_classifier)
    prune.global_unstructured(
        targets, pruning_method=prune.L1Unstructured, amount=amount
    )
    # Report density/sparsity on masked weights
    total, nonzero = 0, 0
    for m, _ in targets:
        w = m.weight  # masked view
        total   += w.numel()
        nonzero += (w != 0).sum().item()
    density  = nonzero / total if total else 1.0
    sparsity = 1.0 - density
    print(f"Global L1 pruning applied: amount={amount:.2f} | density={density:.2%} | sparsity={sparsity:.2%}")
    return targets, density, sparsity

def remove_pruning_masks(targets):
    """Make pruning permanent by removing mask reparametrizations."""
    for m, _ in targets:
        prune.remove(m, "weight")

def state_dict_size_mb(model):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp:
        torch.save(model.state_dict(), tmp.name)
        size_mb = os.path.getsize(tmp.name) / (1024**2)
    os.remove(tmp.name)
    return size_mb

# --- 3) Load baseline model ---
model_pruned = load_fp32_model_state(BEST_FILE, num_labels=len(ORDERED_LABELS))
print(f"Baseline state_dict size (dense): {state_dict_size_mb(model_pruned):.2f} MB")

# --- 4) Prune (recommend starting with 0.2–0.4) ---
targets, density, sparsity = apply_global_l1(model_pruned, amount=0.4, prune_classifier=False)

# (Optional) quick fine-tune to recover accuracy could go here

# --- 5) Make pruning permanent and save ---
remove_pruning_masks(targets)
print(f"Pruned state_dict size (dense save): {state_dict_size_mb(model_pruned):.2f} MB")
torch.save(model_pruned.state_dict(), PRUNED_SD)
print("Saved pruned state_dict to:", PRUNED_SD)

# --- 6) Evaluate pruned model on the SAME DataLoader `dl` ---
# Use your existing evaluate function; CPU is fine
pruned_metrics =eval_int8_cpu(model_pruned, dl)  # or: evaluate_model(model_pruned, dl, device=DEVICE)
print(f"PRUNED -> Acc={pruned_metrics['accuracy']:.4f}, F1={pruned_metrics['f1']:.4f}, "
      f"P={pruned_metrics['precision']:.4f}, R={pruned_metrics['recall']:.4f}")


Baseline state_dict size (dense): 417.73 MB
Global L1 pruning applied: amount=0.40 | density=60.00% | sparsity=40.00%
Pruned state_dict size (dense save): 417.73 MB
Saved pruned state_dict to: /content/drive/MyDrive/deep_learning/HF_best_model_stage3_pruned40_sd.pt
                    precision    recall  f1-score   support

Extremely Negative     0.9030    0.8649    0.8835       592
          Negative     0.8762    0.8223    0.8484      1041
           Neutral     0.7547    0.9095    0.8249       619
          Positive     0.8479    0.8416    0.8447       947
Extremely Positive     0.9014    0.8548    0.8775       599

          accuracy                         0.8531      3798
         macro avg     0.8566    0.8586    0.8558      3798
      weighted avg     0.8575    0.8531    0.8537      3798

PRUNED -> Acc=0.8531, F1=0.8558, P=0.8566, R=0.8586


In [ ]:
# ==== INT8 quantization + evaluation for the TWO BERT models ====
import os, tempfile, torch, torch.nn as nn
from torch.quantization import quantize_dynamic
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report

# Assumes these are already defined earlier in your notebook:
# - MODEL_DIR
# - ordered_labels
# - load_model_for_file(model_file, num_labels, dropout)

# ---------- utilities ----------
import inspect
@torch.inference_mode()
def eval_int8_cpu(model, dataloader, label_names=None):
    """Evaluate on CPU and print per-class report (passes token_type_ids only if supported)."""
    model.eval(); model.cpu()
    # detect which args the model.forward supports
    fwd_params = set(inspect.signature(model.forward).parameters.keys())

    all_preds, all_labels = [], []
    for batch in dataloader:
        ids   = batch["input_ids"]
        mask  = batch.get("attention_mask")
        ttids = batch.get("token_type_ids")

        # build kwargs safely
        kwargs = {"input_ids": ids, "attention_mask": mask}
        if ttids is not None and "token_type_ids" in fwd_params:
            kwargs["token_type_ids"] = ttids

        out = model(**kwargs)
        logits = out["logits"] if isinstance(out, dict) else out.logits
        preds  = torch.argmax(logits, dim=1).cpu().numpy()
        labels = batch.get("labels", batch.get("label")).cpu().numpy()

        all_preds.extend(preds); all_labels.extend(labels)

    # report
    print(classification_report(all_labels, all_preds, target_names=label_names, digits=4))
    metrics = {
        "accuracy":  accuracy_score(all_labels, all_preds),
        "f1":        f1_score(all_labels, all_preds, average="macro"),
        "precision": precision_score(all_labels, all_preds, average="macro", zero_division=0),
        "recall":    recall_score(all_labels, all_preds, average="macro"),
    }
    print(f"Accuracy={metrics['accuracy']:.4f} | F1(macro)={metrics['f1']:.4f} | "
          f"Precision(macro)={metrics['precision']:.4f} | Recall(macro)={metrics['recall']:.4f}")
    return metrics


# ---------- build a BERT-specific test loader (tokenizer must match BERT) ----------
tok_bert = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
coll_bert = DataCollatorWithPadding(tokenizer=tok_bert)

class _EvalBertDS(Dataset):
    def __init__(self, df, text_col="ProcessedTweet", label_col="label", max_len=128):
        self.texts  = df[text_col].astype(str).tolist()
        self.labels = df[label_col].astype(int).tolist()
        self.max_len = max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        enc = tok_bert(self.texts[i], truncation=True, max_length=self.max_len)
        enc["labels"] = self.labels[i]
        return enc

dl_bert = DataLoader(_EvalBertDS(test_df), batch_size=128, shuffle=False, collate_fn=coll_bert)

In [ ]:
# ==== FIX: load with key remapping (encoder. -> backbone. / bert. -> backbone.) ====
import re, os, tempfile, torch, torch.nn as nn
from torch.nn.utils import prune

BEST_FILE = os.path.join(MODEL_DIR, "best_BERT_base_uncasedmodel2.pt")
PRUNED_SD = os.path.join(MODEL_DIR, "best_BERT_base_uncasedmodel2_pruned25_ffn_sd.pt")  # safer name

def _remap_bert_keys(sd: dict) -> dict:
    new_sd = {}
    for k, v in sd.items():
        k2 = k
        if k2.startswith("encoder."):
            k2 = "backbone." + k2[len("encoder."):]
        elif k2.startswith("bert."):
            k2 = "backbone." + k2[len("bert."):]
        # optional old head names:
        k2 = k2.replace(".cls.", ".classifier.")  # if you ever had 'cls' head
        new_sd[k2] = v
    return new_sd

def load_fp32_model_state(path, num_labels):
    model = BertWithDropout("bert-base-uncased", num_labels=num_labels, dropout_rate=0.2)
    raw_sd = torch.load(path, map_location="cpu")
    remapped = _remap_bert_keys(raw_sd)
    missing, unexpected = model.load_state_dict(remapped, strict=False)
    if missing:
        print("WARN missing keys:", missing)
    if unexpected:
        print("WARN unexpected keys:", unexpected)
    model.eval().cpu()
    return model

def state_dict_size_mb(model):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pt") as tmp:
        torch.save(model.state_dict(), tmp.name)
        size_mb = os.path.getsize(tmp.name) / (1024**2)
    os.remove(tmp.name)
    return size_mb

# ==== SAFER PRUNING: FFN-only uniform (BERT) ====
def _collect_ffn_linear_params_only(model, prune_classifier=False):
    """only intermediate.dense / output.dense under backbone.encoder.*"""
    targets = []
    for name, module in model.named_modules():
        if not isinstance(module, nn.Linear):
            continue
        if name.startswith("backbone.encoder"):
            is_ffn = (".intermediate.dense" in name) or (".output.dense" in name)
            if is_ffn:
                targets.append((module, "weight"))
        elif prune_classifier and ("classifier" in name):
            targets.append((module, "weight"))
    return targets

def apply_uniform_l1_ffn(model, amount_per_layer=0.25, prune_classifier=False):
    pairs = _collect_ffn_linear_params_only(model, prune_classifier)
    if not pairs:
        raise RuntimeError("No FFN Linear layers collected for pruning.")
    for m, _ in pairs:
        prune.l1_unstructured(m, name="weight", amount=amount_per_layer)

    total, nonzero = 0, 0
    for m, _ in pairs:
        w = m.weight
        total += w.numel()
        nonzero += (w != 0).sum().item()
    density = nonzero / total if total else 1.0
    sparsity = 1.0 - density
    print(f"Applied FFN-only uniform L1: per-layer amount={amount_per_layer:.2f} | "
          f"density={density:.2%} | sparsity={sparsity:.2%}")
    return pairs

def remove_pruning_masks(targets):
    for m, _ in targets:
        prune.remove(m, "weight")

# ==== RUN ====
model_pruned = load_fp32_model_state(BEST_FILE, num_labels=len(ORDERED_LABELS))
print(f"Baseline state_dict size (dense): {state_dict_size_mb(model_pruned):.2f} MB")


targets = apply_uniform_l1_ffn(model_pruned, amount_per_layer=0.4, prune_classifier=False)# use safer FFN-only 0.25 first (you can try 0.20–0.25)

remove_pruning_masks(targets)
print(f"Pruned state_dict size (dense save): {state_dict_size_mb(model_pruned):.2f} MB")
torch.save(model_pruned.state_dict(), PRUNED_SD)
print("Saved pruned state_dict to:", PRUNED_SD)

# ==== EVALUATE on your existing test loader ====
# Make sure you have evaluate_model_cpu that accepts token_type_ids safely (or just use yours)
pruned_metrics = eval_int8_cpu(model_pruned, test_loader_clean, label_names=ORDERED_LABELS)
print(f"PRUNED -> Acc={pruned_metrics['accuracy']:.4f}, F1={pruned_metrics['f1']:.4f}, "
      f"P={pruned_metrics['precision']:.4f}, R={pruned_metrics['recall']:.4f}")


Baseline state_dict size (dense): 417.73 MB
Applied FFN-only uniform L1: per-layer amount=0.40 | density=60.00% | sparsity=40.00%
Pruned state_dict size (dense save): 417.73 MB
Saved pruned state_dict to: /content/drive/MyDrive/deep_learning/best_BERT_base_uncasedmodel2_pruned25_ffn_sd.pt
                    precision    recall  f1-score   support

Extremely Negative     0.8756    0.8801    0.8778       592
          Negative     0.8508    0.8598    0.8552      1041
           Neutral     0.9350    0.8595    0.8956       619
          Positive     0.8285    0.8363    0.8324       947
Extremely Positive     0.8482    0.8865    0.8669       599

          accuracy                         0.8612      3798
         macro avg     0.8676    0.8644    0.8656      3798
      weighted avg     0.8624    0.8612    0.8615      3798

Accuracy=0.8612 | F1(macro)=0.8656 | Precision(macro)=0.8676 | Recall(macro)=0.8644
PRUNED -> Acc=0.8612, F1=0.8656, P=0.8676, R=0.8644


# Knowledge Distillation Demo

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, DataCollatorWithPadding

device = "cuda" if torch.cuda.is_available() else "cpu"

# ===== Teacher: BERT base (Stage 3) =====
# If you already have a loader, replace this with it.
def load_fp32_teacher(path, num_labels):
    model = BertWithDropout("bert-base-uncased", num_labels=num_labels, dropout_rate=0.2)
    state = torch.load(path, map_location="cpu")          # expects state_dict
    model.load_state_dict(state, strict=True)
    model.eval()                                          # eval mode (dropout off)
    for p in model.parameters():
        p.requires_grad = False                           # fully frozen teacher
    return model

teacher_ckpt =data_dir /"HF_best_model_stage3.pt"  # STAGE 3
teacher = load_fp32_teacher(teacher_ckpt, num_labels=len(ORDERED_LABELS)).to(device)

# ===== Student: DistilBERT =====
student = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=len(ORDERED_LABELS)
).to(device)

# (Optional, only if you need a tokenizer/collator here)
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased", use_fast=True)
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ===== Custom Trainer for KD =====
class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, temperature=2.0, alpha=0.7, **kwargs):
        """
        alpha = weight for CE (hard labels); (1-alpha) = weight for KD (soft labels).
        temperature controls softening of distributions.
        """
        super().__init__(*args, **kwargs)
        assert teacher_model is not None, "teacher_model must be provided"
        self.teacher = teacher_model
        self.temperature = temperature
        self.alpha = alpha

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Student forward (Trainer already moved inputs to device)
        outputs_student = model(**inputs)
        logits_student = outputs_student.logits

        # Teacher forward (no grad, no token_type_ids)
        with torch.no_grad():
            t_outputs = self.teacher(
                input_ids=inputs["input_ids"],
                attention_mask=inputs.get("attention_mask", None)
            )
            logits_teacher = t_outputs["logits"] if isinstance(t_outputs, dict) else t_outputs.logits

        # Hard-label loss (ground truth)
        loss_ce = F.cross_entropy(logits_student, inputs["labels"])

        # Soft-label loss (teacher distillation)
        T = self.temperature
        loss_kl = F.kl_div(
            F.log_softmax(logits_student / T, dim=-1),
            F.softmax(logits_teacher / T, dim=-1),
            reduction="batchmean"
        ) * (T ** 2)

        loss = self.alpha * loss_ce + (1.0 - self.alpha) * loss_kl
        return (loss, outputs_student) if return_outputs else loss

# ===== Training Args =====
training_args = TrainingArguments(
    output_dir="./distill_results_stage3_distilbert",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    eval_strategy="epoch",           # <-- corrected
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",            # must match compute_metrics_fn keys
    greater_is_better=True,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    fp16=torch.cuda.is_available(),        # speed on GPU
    report_to="none",
)

# ===== Trainer =====
trainer_distill = DistillationTrainer(
    model=student,
    teacher_model=teacher,
    args=training_args,
    train_dataset=ds_tok["train"],     # must return input_ids/attention_mask/labels
    eval_dataset=ds_tok["test"],
    compute_metrics=compute_metrics_fn,
    # tokenizer=tokenizer,             # uncomment if you want Trainer-owned padding
    # data_collator=data_collator,
)

# ===== Train =====
trainer_distill.train()

# ===== Save student =====
trainer_distill.save_model("./distill_results_stage3_distilbert/best_student")
print("\nDistillation complete. Student (DistilBERT) model saved.")
print("Student parameter count:", sum(p.numel() for p in student.parameters()))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Micro,F1 Macro,F1 Weighted
1,0.998000,0.914319,0.802791,0.802791,0.811175,0.802942
2,0.704500,0.819215,0.822012,0.822012,0.829159,0.822848
3,0.437000,0.805700,0.838863,0.838863,0.846104,0.839032
4,0.277500,0.827808,0.838073,0.838073,0.842795,0.838088
5,0.215200,0.880881,0.840706,0.840706,0.844729,0.840443



Distillation complete. Student (DistilBERT) model saved.
Student parameter count: 66957317


In [ ]:
# --- Drop-in patch: handles models that don't accept token_type_ids (e.g., DistilBERT) ---
import inspect
import torch, numpy as np
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

def _get_logits(outputs):
    return outputs["logits"] if isinstance(outputs, dict) else outputs.logits

def _supports_token_type_ids(model):
    try:
        return "token_type_ids" in inspect.signature(model.forward).parameters
    except (ValueError, AttributeError):
        return False

@torch.inference_mode()
def eval_student(model, dataloader, device="cpu", label_names=None):
    """Evaluate a model on a DataLoader. Safe for both HF (no token_type_ids) and custom BERT (with token_type_ids)."""
    model.eval(); model.to(device)
    all_preds, all_labels = [], []
    use_ttids = _supports_token_type_ids(model)

    for batch in dataloader:
        ids   = batch["input_ids"].to(device)
        mask  = batch.get("attention_mask")
        if mask is not None: mask = mask.to(device)
        ttids = batch.get("token_type_ids")
        if ttids is not None: ttids = ttids.to(device)
        labels = batch.get("labels", batch.get("label")).to(device)

        kwargs = {"input_ids": ids, "attention_mask": mask}
        if use_ttids and ttids is not None:
            kwargs["token_type_ids"] = ttids

        outputs = model(**kwargs)
        preds = torch.argmax(_get_logits(outputs), dim=-1)

        all_preds.append(preds.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

    y_pred = np.concatenate(all_preds); y_true = np.concatenate(all_labels)

    print("=== Classification Report ===")
    print(classification_report(y_true, y_pred, target_names=label_names, digits=4) if label_names
          else classification_report(y_true, y_pred, digits=4))

    metrics = {
        "accuracy":  accuracy_score(y_true, y_pred),
        "f1":        f1_score(y_true, y_pred, average="macro"),
        "precision": precision_score(y_true, y_pred, average="macro", zero_division=0),
        "recall":    recall_score(y_true, y_pred, average="macro"),
    }
    print(f"Accuracy={metrics['accuracy']:.4f} | F1(macro)={metrics['f1']:.4f} | "
          f"Precision(macro)={metrics['precision']:.4f} | Recall(macro)={metrics['recall']:.4f}")
    return metrics


In [ ]:
# Student FP32 (after KD)
stu_device = "cuda" if torch.cuda.is_available() else "cpu"
stu_metrics = eval_student(student, dl, device=stu_device, label_names=ORDERED_LABELS)


=== Classification Report ===
                    precision    recall  f1-score   support

Extremely Negative     0.8433    0.8818    0.8621       592
          Negative     0.8252    0.8165    0.8209      1041
           Neutral     0.9190    0.8433    0.8795       619
          Positive     0.7864    0.8046    0.7954       947
Extremely Positive     0.8480    0.8664    0.8571       599

          accuracy                         0.8360      3798
         macro avg     0.8444    0.8425    0.8430      3798
      weighted avg     0.8372    0.8360    0.8362      3798

Accuracy=0.8360 | F1(macro)=0.8430 | Precision(macro)=0.8444 | Recall(macro)=0.8425


In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from transformers import (
    AutoTokenizer, DataCollatorWithPadding,
    AutoModelForSequenceClassification, Trainer, TrainingArguments, TrainerCallback
)

device = "cuda" if torch.cuda.is_available() else "cpu"
NUM_LABELS = len(ORDERED_LABELS)

# ----------------------------
# Tokenizer/Collator (student)
# ----------------------------
STUDENT_CKPT = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(STUDENT_CKPT, use_fast=True)
collator  = DataCollatorWithPadding(tokenizer=tokenizer)

# ----------------------------
# Dataset that uses the tokenizer above
# ----------------------------
class TextClsDataset(Dataset):
    def __init__(self, df, text_col: str, label_col: str, max_len: int = 128):
        self.texts  = df[text_col].astype(str).tolist()
        self.labels = df[label_col].astype(int).tolist()
        self.max_len = max_len
    def __len__(self): return len(self.texts)
    def __getitem__(self, i):
        enc = tokenizer(self.texts[i], truncation=True, max_length=self.max_len)
        enc["labels"] = self.labels[i]
        return enc

# Build train/val/test datasets (adjust text column if needed)
train_ds = TextClsDataset(train_df, text_col="ProcessedTweet", label_col="label")
val_ds   = TextClsDataset(eval_df,   text_col="ProcessedTweet", label_col="label")
test_ds  = TextClsDataset(test_df,   text_col="ProcessedTweet", label_col="label")

# ----------------------------
# Teacher loader with key remapping
# ----------------------------
def _remap_bert_keys(sd: dict) -> dict:
    """Map checkpoint keys saved as 'encoder.*' or 'bert.*' to 'backbone.*'.
       Also strips 'module.' if present, and normalizes '.cls.' -> '.classifier.'.
    """
    new_sd = {}
    for k, v in sd.items():
        k2 = k
        if k2.startswith("module."):
            k2 = k2[len("module."):]
        if k2.startswith("encoder."):
            k2 = "backbone." + k2[len("encoder."):]
        elif k2.startswith("bert."):
            k2 = "backbone." + k2[len("bert."):]
        k2 = k2.replace(".cls.", ".classifier.")
        new_sd[k2] = v
    return new_sd

def load_fp32_teacher(path, num_labels):
    model = BertWithDropout("bert-base-uncased", num_labels=num_labels, dropout_rate=0.2)
    raw_sd = torch.load(path, map_location="cpu")
    remapped = _remap_bert_keys(raw_sd)
    missing, unexpected = model.load_state_dict(remapped, strict=False)
    if missing:    print("[WARN] missing keys:", missing)
    if unexpected: print("[WARN] unexpected keys:", unexpected)
    model.eval()
    for p in model.parameters():
        p.requires_grad = False
    return model

teacher_ckpt = data_dir/"best_BERT_base_uncasedmodel2.pt"
teacher = load_fp32_teacher(teacher_ckpt, num_labels=NUM_LABELS).to(device)

# ----------------------------
# Student: DistilBERT
# ----------------------------
student = AutoModelForSequenceClassification.from_pretrained(
    STUDENT_CKPT, num_labels=NUM_LABELS
).to(device)

# ----------------------------
# Metrics (adds micro/weighted)
# ----------------------------
def compute_metrics_fn(eval_pred):
    preds  = getattr(eval_pred, "predictions", eval_pred[0])
    labels = getattr(eval_pred, "label_ids",    eval_pred[1])
    if isinstance(preds, tuple): preds = preds[0]
    preds = np.argmax(preds, axis=-1)
    return {
        "accuracy":      accuracy_score(labels, preds),
        "f1_macro":      f1_score(labels, preds, average="macro"),
        "f1_micro":      f1_score(labels, preds, average="micro"),
        "f1_weighted":   f1_score(labels, preds, average="weighted"),
        "precision":     precision_score(labels, preds, average="macro", zero_division=0),
        "recall":        recall_score(labels, preds, average="macro"),
    }

# ----------------------------
# KD Trainer
# ----------------------------
class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, temperature=2.0, alpha=0.7, **kwargs):
        super().__init__(*args, **kwargs)
        assert teacher_model is not None, "teacher_model must be provided"
        self.teacher = teacher_model
        self.temperature = temperature
        self.alpha = alpha

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # DistilBERT ignores token_type_ids; teacher BERT can use them if present
        s_inputs = dict(inputs)
        s_inputs.pop("token_type_ids", None)

        out_s = model(**s_inputs)
        logits_student = out_s.logits

        with torch.no_grad():
            t_out = self.teacher(
                input_ids=s_inputs["input_ids"],
                attention_mask=s_inputs.get("attention_mask"),
                token_type_ids=inputs.get("token_type_ids")  # safe: None if not present
            )
            logits_teacher = t_out["logits"] if isinstance(t_out, dict) else t_out.logits

        loss_ce = F.cross_entropy(logits_student, inputs["labels"])

        T = self.temperature
        loss_kl = F.kl_div(
            F.log_softmax(logits_student / T, dim=-1),
            F.softmax(logits_teacher / T, dim=-1),
            reduction="batchmean"
        ) * (T ** 2)

        loss = self.alpha * loss_ce + (1.0 - self.alpha) * loss_kl
        return (loss, out_s) if return_outputs else loss

# ----------------------------
# Pretty per-epoch table (optional but handy)
# ----------------------------
class EpochTableCallback(TrainerCallback):
    def __init__(self):
        self._printed_header = False
        self.rows = []
    def _last_train_loss_for_epoch(self, state, epoch_int):
        for rec in reversed(state.log_history):
            if rec.get("epoch") is None: continue
            if int(round(rec["epoch"])) == epoch_int and "loss" in rec:
                return float(rec["loss"])
        return float("nan")
    def on_epoch_end(self, args, state, control, **kwargs):
        control.should_evaluate = True
    def on_evaluate(self, args, state, control, metrics, **kwargs):
        from math import isnan
        ep = int(round(metrics.get("epoch", state.epoch or 0))) or (len(self.rows) + 1)
        tr_loss = self._last_train_loss_for_epoch(state, ep)
        row = {
            "Epoch": ep,
            "Training Loss": tr_loss,
            "Validation Loss": float(metrics.get("eval_loss", "nan")),
            "Accuracy": float(metrics.get("eval_accuracy", "nan")),
            "F1 Micro": float(metrics.get("eval_f1_micro", "nan")),
            "F1 Macro": float(metrics.get("eval_f1_macro", "nan")),
            "F1 Weighted": float(metrics.get("eval_f1_weighted", "nan")),
        }
        self.rows.append(row)
        if not self._printed_header:
            print("\nEpoch  Training Loss  Validation Loss   Accuracy   F1 Micro   F1 Macro  F1 Weighted")
            self._printed_header = True
        def f(x):
            return "   nan" if (x != x) or isnan(x) else f"{x:10.6f}"
        print(f"{row['Epoch']:>5}  {f(row['Training Loss'])}   {f(row['Validation Loss'])}  "
              f"{f(row['Accuracy'])}  {f(row['F1 Micro'])}  {f(row['F1 Macro'])}  {f(row['F1 Weighted'])}")

epoch_table = EpochTableCallback()

# ----------------------------
# TrainingArguments
# ----------------------------
training_args = TrainingArguments(
    output_dir="./distill_results_best_BERT_base_uncasedmodel2",
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    eval_strategy="epoch",   # <- correct arg name
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro",
    greater_is_better=True,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    fp16=torch.cuda.is_available(),
    report_to="none",
    remove_unused_columns=False,
)

# ----------------------------
# Trainer
# ----------------------------
trainer_distill = DistillationTrainer(
    model=student,
    teacher_model=teacher,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    compute_metrics=compute_metrics_fn,
    data_collator=collator,
)

trainer_distill.add_callback(epoch_table)

# ----------------------------
# Train
# ----------------------------
trainer_distill.train()

# ----------------------------
# End-of-run Classification Report on TEST
# ----------------------------
pred = trainer_distill.predict(test_ds)
y_true = pred.label_ids
y_pred = pred.predictions.argmax(axis=-1)
print("\n=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=ORDERED_LABELS, digits=4))

# ----------------------------
# Save student
# ----------------------------
trainer_distill.save_model("./distill_results_best_BERT_base_uncasedmodel2/best_student")
print("\nDistillation complete. Student (DistilBERT) model saved.")
print("Student parameter count:", sum(p.numel() for p in student.parameters()))


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Precision,Recall
1,0.776300,0.616122,0.827017,0.832538,0.827017,0.826715,0.828593,0.840117
2,0.522000,0.546772,0.853134,0.856823,0.853134,0.853718,0.870272,0.851172
3,0.290900,0.515430,0.860058,0.863588,0.860058,0.860517,0.873242,0.859110
4,0.223400,0.471406,0.875364,0.878759,0.875364,0.875434,0.878200,0.880345
5,0.131100,0.497551,0.875729,0.878367,0.875729,0.875666,0.875760,0.881717



Epoch  Training Loss  Validation Loss   Accuracy   F1 Micro   F1 Macro  F1 Weighted
    1    0.776300     0.616122    0.827017    0.827017    0.832538    0.826715
    2    0.522000     0.546772    0.853134    0.853134    0.856823    0.853718
    3    0.290900     0.515430    0.860058    0.860058    0.863588    0.860517
    4    0.223400     0.471406    0.875364    0.875364    0.878759    0.875434
    5    0.131100     0.497551    0.875729    0.875729    0.878367    0.875666



=== Classification Report ===
                    precision    recall  f1-score   support

Extremely Negative     0.8535    0.9155    0.8835       592
          Negative     0.8617    0.8261    0.8436      1041
           Neutral     0.9135    0.8530    0.8822       619
          Positive     0.8084    0.8553    0.8312       947
Extremely Positive     0.8821    0.8614    0.8716       599

          accuracy                         0.8573      3798
         macro avg     0.8638    0.8623    0.8624      3798
      weighted avg     0.8588    0.8573    0.8574      3798


Distillation complete. Student (DistilBERT) model saved.
Student parameter count: 66957317
